In [8]:
import pandas as pd
import time
import json
import pycountry as pc
import requests

ModuleNotFoundError: No module named 'pandas'

## 下载文件

In [9]:
r = requests.get("https://coronavirus-tracker-api.herokuapp.com/all") 
with open("raw/all.json", "wb") as f:
    f.write(r.content)

NameError: name 'requests' is not defined

## 读取文件

In [10]:
with open('raw/all.json','r',encoding='utf8')as fp:
    json_data = json.load(fp)
    print(json_data.keys())
    print(type(json_data))

NameError: name 'json' is not defined

In [11]:
json_data['confirmed'].keys()

NameError: name 'json_data' is not defined

In [12]:
confirmed = json_data['confirmed']['locations']
deaths = json_data['deaths']['locations']
recovered = json_data['recovered']['locations']

NameError: name 'json_data' is not defined

In [13]:
print(len(confirmed), len(deaths), len(recovered))

NameError: name 'confirmed' is not defined

## 中国各省

In [14]:
china_indexes = []
for (index, item) in enumerate(json_data['confirmed']['locations']):
    if item['country'] == 'China':
        china_indexes.append(index)
print(china_indexes)

NameError: name 'json_data' is not defined

In [ ]:
provinces = []
for i in china_indexes:
    p_dict = {}
    p_dict['country'] = confirmed[i]['country']
    p_dict['country_code'] = pc.countries.get(
        alpha_2=confirmed[i]['country_code']).alpha_3
    p_dict['province'] = confirmed[i]['province']
    p_dict['coordinates'] = confirmed[i]['coordinates']
    p_dict['latest'] = {
        'confirmed': confirmed[i]['latest'],
        'deaths': deaths[i]['latest'],
        'recovered': recovered[i]['latest']
    }

    timeline = []
    for date in confirmed[i]['history'].keys():
        date_ = time.strftime("%Y-%m-%d", time.strptime(date, "%m/%d/%y"))
        #     print({'date': date_, 'confirmed': cnt})
        timeline.append({
            'date': date_,
            'confirmed': confirmed[i]['history'][date],
            'deaths': deaths[i]['history'][date],
            'recovered': recovered[i]['history'][date]
        })
    p_dict['timeline'] = timeline

    provinces.append(p_dict)

In [ ]:
china_result = pd.DataFrame(provinces)

In [ ]:
china_result.to_json("china_timeline.json", orient='records')

## 世界各国

In [ ]:
DF_confirmed_groups = pd.DataFrame(confirmed).groupby('country')
DF_deaths_groups = pd.DataFrame(deaths).groupby('country')
DF_recovered_groups = pd.DataFrame(recovered).groupby('country')

In [ ]:
# pd.DataFrame(confirmed).country.drop_duplicates().tolist()

In [ ]:
print(len(DF_confirmed_groups), len(DF_deaths_groups), len(DF_recovered_groups))

In [ ]:
cntry_names = list(DF_confirmed_groups.groups.keys())

In [ ]:
# DF_confirmed_groups.get_group('China')

In [ ]:
countries = []
for cntry in cntry_names:
    df = DF_confirmed_groups.get_group(cntry)
    df_deaths = DF_deaths_groups.get_group(cntry)
    df_recovered = DF_recovered_groups.get_group(cntry)

    dates = list(df.history.iloc[0].keys())
    timeline = []

    for d in dates:
        date_ = time.strftime("%Y-%m-%d", time.strptime(d, "%m/%d/%y"))

        # confirmed each day
        sum_confirmed = 0
        for h in df.history:
            sum_confirmed += h[d]

        # deaths each day
        sum_deaths = 0
        for h in df_deaths.history:
            sum_deaths += h[d]

        # recovered each day
        sum_recovered = 0
        for h in df_recovered.history:
            sum_recovered += h[d]

        timeline.append({
            'date': date_,
            'confirmed': sum_confirmed,
            'deaths': sum_deaths,
            'recovered': sum_recovered
        })
    
    # The cruise ship Diamond Princess has no country code, marked as 'XXX'
    try:
        country_code_3 = pc.countries.get(alpha_2=df.iloc[0].country_code).alpha_3
    except:
        country_code_3 = 'XXX'
        
    countries.append({
        'country':
        cntry,
        'country_code':
        country_code_3,
        'latest': {
            'confirmed': sum_confirmed,
            'deaths': sum_deaths,
            'recovered': sum_recovered
        },
        'timeline':
        timeline
    })

In [ ]:
countries_result = pd.DataFrame(countries)

In [ ]:
countries_result.to_json("world_timeline.json", orient='records')

## We are the world

In [ ]:
countries_result.head()

In [ ]:
global_confirmed, global_deaths, global_recovered = 0, 0, 0
for entry in countries_result.latest:
    global_confirmed += entry['confirmed']
    global_deaths += entry['deaths']
    global_recovered += entry['recovered']

global_latest = {
    'confirmed': global_confirmed,
    'deaths': global_deaths,
    'recovered': global_recovered
}
print(global_latest)

In [ ]:
global_timeline = countries_result.iloc[0].timeline
print("Data in {} days.".format(len(global_timeline)))
for i in range(len(global_timeline)):
    global_daily_confirmed = 0
    global_daily_deaths = 0
    global_daily_recovered = 0
    for tl in countries_result.timeline:
        global_daily_confirmed += tl[i]['confirmed']
        global_daily_deaths += tl[i]['deaths']
        global_daily_recovered += tl[i]['recovered']
    global_timeline[i]['confirmed'] = global_daily_confirmed
    global_timeline[i]['deaths'] = global_daily_deaths
    global_timeline[i]['recovered'] = global_daily_recovered
print(global_timeline)

In [ ]:
global_data = {
    'country': 'global',
    'latest': global_latest,
    'timeline': global_timeline
}
with open('global_timeline.json', 'w') as f:
    json.dump(global_data, f)